# **Esquema de comunicación con Codificación Hash**

### **Importamos las librerias**

In [216]:
import hashlib
import random
from bitstring import BitArray
from collections import Counter

### **1. Fuente de información**

In [217]:
def leer_archivo(nombre_archivo):
    try:
        with open(nombre_archivo, 'r') as archivo:
            contenido = archivo.read()
            return contenido
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return ""

### **2. Transmisor**

In [218]:
def Convertir_a_Lista(mensaje):
  lista = []
  for i in mensaje:
    lista.append(i)
  return lista

In [219]:
def Convertir_a_String(lista):
    return ''.join(lista)

In [220]:
def Binario_a_Texto(binario):
    bloques = [binario[i:i+8] for i in range(0, len(binario), 8)]
    caracteres = [chr(int(bloque, 2)) for bloque in bloques]
    texto = ''.join(caracteres)
    return texto

In [221]:
def Texto_a_Binario(mensaje):
    bitsM = BitArray(bytes=mensaje.encode('utf-8'))
    return bitsM.bin

#### **Codificación Run Lengh Enconding**

In [222]:
def CodificacionRunLengEnconding(cadena):
  RunLengEnconding = []
  contador = 1
  simbolo = cadena[0]
  RunLengEnconding.append(cadena[0])
  for i in range(1,len(cadena)):
    if(simbolo == cadena[i]):
        contador = contador + 1
    else:
        RunLengEnconding.append(contador)
        RunLengEnconding.append(cadena[i])
        contador = 1
        simbolo = cadena[i]

  RunLengEnconding.append(contador)
  Elemento = [str(elemento) for elemento in RunLengEnconding]

  cadenaRLE = ''.join(Elemento)
  return cadenaRLE

#### **Codificación Hash**

In [223]:
def FrecuenciasRelativas(lista_patrones):
  contador = Counter(lista_patrones)
  total_palabras = len(lista_patrones)
  palabras = []
  frecuencias_relativas = []

  for palabra, count in contador.items():
      palabras.append(palabra)
      frecuencia_relativa = count / total_palabras
      frecuencias_relativas.append(frecuencia_relativa)
  palabras_ordenadas, frecuencias_ordenadas = zip(*sorted(zip(palabras, frecuencias_relativas), key=lambda x: x[1], reverse=True))

  palabras_ordenadas = list(palabras_ordenadas)
  frecuencias_ordenadas = list(frecuencias_ordenadas)

  for i in range(0,len(frecuencias_ordenadas)-1):
    suma =1e-12
    for j in range(i,len(frecuencias_ordenadas)-1):
      if(frecuencias_ordenadas[i] == frecuencias_ordenadas[j+1]):
        frecuencias_ordenadas[j+1] = frecuencias_ordenadas[j+1] - suma
        suma = suma + 1e-12
      else:
        break
  return palabras_ordenadas, frecuencias_ordenadas

In [224]:
def Transmisor(BinariList):
  patrones, frecuencias = FrecuenciasRelativas(BinariList)
  lista_RLE = []

  for i in patrones:
    lista_RLE.append(CodificacionRunLengEnconding(i))
  return dict(zip(patrones,lista_RLE))

In [225]:
def CodificarLista(BinariList,handshake):
  codificar = []
  for i in BinariList:
    codificar.append(handshake[i])
  return codificar

In [226]:
def BorrarAleatorio(lista1, lista2):
    if lista1 and lista2:
        indice_aleatorio = random.randint(0, min(len(lista1), len(lista2)) - 1)
        lista1.pop(indice_aleatorio)
        valor_eliminado = lista2[indice_aleatorio]
        lista2.pop(indice_aleatorio)
        return lista1,lista2,valor_eliminado

### **3. Canal**

In [227]:
def Canal(paquete,indices,canales):
  verdad = False
  verdad_ruido = False
  canal_usado = None
  indice_eliminado = None
  for i in range(0,len(canales)):
    if(canales[i][0] == 0):
      verdad = True
      canal_usado = i
      if(random.randint(0, 1)):
        canales[i][0] = 1
      canales[i][1] = paquete
      canales[i][2] = indices
      if(canales[i][0] == 1):
        verdad_ruido = True
        canales[i][1], canales[i][2],indice_eliminado = BorrarAleatorio(canales[i][1],canales[i][2])
        for j in range(0,len(canales)):
          canales[j][0] = 0
      break

  return canales,verdad_ruido,canal_usado,indice_eliminado

### **Busqueda Binaria**

In [228]:
def BusquedaBinaria(lista_valores, buscar):
    inicio = 0
    fin = len(lista_valores) - 1

    while inicio <= fin:
        medio = (inicio + fin) // 2

        if lista_valores[medio] == buscar:
            return medio
        if buscar < lista_valores[medio]:
            fin = medio - 1
        else:
            inicio = medio + 1
    return -1

In [229]:
def Ordenar(diccionario):
  claves = [str(clave) for clave in diccionario.keys()]
  valores = [str(valor) for valor in diccionario.values()]
  claves_y_valores = list(zip(claves, valores))
  claves_ordenadas = [clave for clave, valor in sorted(claves_y_valores, key=lambda x: x[1])]
  valores.sort()
  return claves_ordenadas,valores

### **Genear Hash**

Con ayuda de la libreria "hashlib"

In [230]:
def GenerarHash(dato):
  bdatos = bytes(dato,'utf-8')
  h = hashlib.new("sha256",bdatos)
  digest=h.hexdigest()
  return digest

### **Comparar Hash**

In [231]:
def CompararHash(cadenaList,claves,valores):
  indice = BusquedaBinaria(valores, cadenaList)

  if(indice != -1):
    return claves[indice]
  else:
    return "x"

In [232]:
def Decoficar(Lista_codificada,handshake):
  decodificar = []
  diccionario_hash = {clave: GenerarHash(valor) for clave, valor in handshake.items()}
  claves_ordenados,valores_ordenados = Ordenar(diccionario_hash)

  for i in Lista_codificada:
    decodificar.append(CompararHash(i,claves_ordenados,valores_ordenados))
  return decodificar

### **4. Receptor**

In [233]:
def Receptor(mensaje_codificado,reglas):
  canales_usado = [[0,None,None],[0,None,None],[0,None,None],[0,None,None]]
  paquete = []
  indices = []
  recibe_paquetes = lista_nula = [None] * len(mensaje_codificado)
  tamaño = len(mensaje_codificado)
  contador = 0
  suma = 4

  while contador < tamaño:
    for i in range(contador,contador+suma):
      if i != (tamaño):
        paquete.append(mensaje_codificado[i])
        indices.append(i)
      else:
        contador = contador + suma
        break

    contador = contador + suma
    canales_usado,verdad_ruido,canal_usado,indice_eliminado = Canal(paquete,indices,canales_usado)
    paquete = []
    indices = []

    if indice_eliminado == None:
      suma = 4
    else:
      suma = 3
      paquete.append(mensaje_codificado[indice_eliminado])
      indices.append(indice_eliminado)


    valores_obtenido = canales_usado[canal_usado][1]
    indices_de_valores = canales_usado[canal_usado][2]

    for valorEnviado, indiceEnviado in zip(valores_obtenido, indices_de_valores):
      recibe_paquetes[indiceEnviado] = valorEnviado

  if paquete:
    recibe_paquetes[indices[0]] = paquete[0]
  decodificar = Decoficar(recibe_paquetes,reglas)
  return decodificar

### **5. Destino de información**

In [234]:
nombre_archivo_fuente = "input.txt"
texto_original = leer_archivo(nombre_archivo_fuente)

print("Mensaje enviado:",texto_original)

Lista = Convertir_a_Lista(texto_original)
Binario = [Texto_a_Binario(caracter) for caracter in Lista]
HandShake = Transmisor(Binario)
Codificacion =  CodificarLista(Binario, HandShake)
Hash = [GenerarHash(cadena) for cadena in Codificacion]

print("\nMensaje codificado:", Codificacion)
print("\nMensaje con SHA256:", Hash)
print("")
print("|--------------------------|")
print("|          Simbolos        |")
print("|--------------------------|")

for clave, valor in HandShake.items():
    print(f"| {clave} | {valor}")
print("|--------------------------|")

texto_decodificado = Receptor(Hash,HandShake)
mi_cadena = Convertir_a_String(texto_decodificado)
texto_resultante = Binario_a_Texto(mi_cadena)

print("\nMensaje recivido:",texto_resultante)

with open("output.txt", "w", encoding="utf-8") as archivo_salida:
    archivo_salida.write(texto_resultante)
print()
print("Mensaje decodificado guardado en 'output.txt'.")

Mensaje enviado: Hola, este es un mensaje de prueba para la comunicacion.

Mensaje codificado: ['0111021103', '01120114', '0112011202', '01120411', '0211011202', '021105', '011202110111', '01130212', '0113011102', '011202110111', '021105', '011202110111', '01130212', '021105', '011301110111', '0112011301', '021105', '011201120111', '011202110111', '0112011301', '01130212', '01120411', '01120111011101', '011202110111', '021105', '0112021102', '011202110111', '021105', '011304', '0113021101', '011301110111', '011202110111', '0112031101', '01120411', '021105', '011304', '01120411', '0113021101', '01120411', '021105', '0112011202', '01120411', '021105', '01120312', '01120114', '011201120111', '011301110111', '0112011301', '011201110211', '01120312', '01120411', '01120312', '011201110211', '01120114', '0112011301', '0211011301']

Mensaje con SHA256: ['a70ca45d5e91b2b1526b41ce0ffc58b83af7befdfed4ed5fb6cb07b0914e821b', '78c0cb3864f477241ec25d5d12bdb52d5c1c2bd99ef73d26d5530d0933cbaa04', '0137f